In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("data/attacks.csv",encoding="Latin-1")

In [3]:
print(df.shape)
print(list(df.columns))

(25723, 24)
['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


## HIPÓTESIS: SON ATACADAS MAS PERSONAS QUE PRATICAN SURF EN FLORIDA(USA) POR TIBURÓN EN LA TEMPORADA PRIMAVERA-VERANO QUE EN LA TEMPORADA OTOÑO-INVIERNO

#### REDUZCO EL NÚMERO DE COLUMNAS A LAS QUE ME INTERESAN PARA MI ESTUDIO

In [4]:
df_clean=df[['Date','Country', 'Area','Activity','Location']]

In [5]:
df_clean.head()

,Date,Country,Area,Activity,Location
0,25-Jun-2018,USA,California,Paddling,"Oceanside, San Diego County"
1,18-Jun-2018,USA,Georgia,Standing,"St. Simon Island, Glynn County"
2,09-Jun-2018,USA,Hawaii,Surfing,"Habush, Oahu"
3,08-Jun-2018,AUSTRALIA,New South Wales,Surfing,Arrawarra Headland
4,04-Jun-2018,MEXICO,Colima,Free diving,La Ticla


In [6]:
df_clean.shape

(25723, 5)

#### COMPRUEBO SI EXISTEN VALORES NULOS EN LA COLUMNA "DATE" Y PROCEDO A ELIMINARLOS YA QUE NO SON VÁLIDOS PARA MI ESTUDIO

In [7]:
df_clean.Date.isnull().sum()


19421

In [8]:
df_clean.drop(df_clean[df_clean.Date.isnull()].index, inplace = True)

In [9]:
df_clean.shape

(6302, 5)

#### PRIMER BARRIDO DE REGISTROS DUPLICADOS

In [10]:
df_clean.duplicated().sum()

98

In [11]:
df_clean.drop(df_clean[df_clean.duplicated()].index, inplace = True)

In [12]:
# Compruebo que los registros se eliminan 
df_clean.duplicated().sum()


0

In [13]:
df_clean.shape

(6204, 5)

#### ESTUDIO DE LOS VALORES NULOS EN EL DATAFRAME

In [14]:
df_clean.isnull().sum()


Date          0
Country      49
Area        449
Activity    536
Location    529
dtype: int64

In [15]:
#Intento sacar información con la que pueda rellenar valores nulos

In [16]:
def search(df,column1,column2,expression):
    return df[(df[column1].isnull()) & (df[column2].str.contains(expression,regex=True))]

In [17]:
search(df_clean,"Area","Location",r"\bFlorida\b")
# Obtengo información. Sin embargo, mi estudio se centra en surfistas, por lo que lo obvio


,Date,Country,Area,Activity,Location
3605,21-Oct-1965,NaN,NaN,The boat Caribou II sank,Florida Strait
4026,Early summer 1960,CARIBBEAN SEA,NaN,Aircraft crashed into sea,"Between Key West, Florida & Guantanamo Bay, Cuba"


In [18]:
search(df_clean,"Area","Country",r"\bFlorida\b")
# No obtengo información 

,Date,Country,Area,Activity,Location


In [19]:
search(df_clean,"Country","Area",r"\b(?:USA|US)\b")
# No obtengo información

,Date,Country,Area,Activity,Location


In [20]:
search(df_clean,"Country","Location",r"\b(?:USA|US)\b")
# Obtengo información. Sin embargo, mi estudio se centra en surfistas, por lo que lo obvio


,Date,Country,Area,Activity,Location


In [21]:
# Elimino registros que son nulos

In [22]:
df_clean=df_clean.dropna()

In [23]:
df_clean.shape

(5033, 5)

In [24]:
# Compruebo que no existen mas valores nulos

df_clean.isnull().sum()

Date        0
Country     0
Area        0
Activity    0
Location    0
dtype: int64

#### ELIMINO LA COLUMNA DE "LOCATION" YA QUE LA HE USADO PARA INTENTAR OBTENER INFORMACIÓN PERO NO LA NECESITO DE NUEVO EN MI ESTUDIO

In [25]:
df_clean=df_clean.drop(["Location"], axis=1)

In [26]:
df_clean.shape

(5033, 4)

#### REDUZCO EL DATAFRAME A LAS ACTIVIDA DE "SURFING" EN FLORIDA (USA)

In [27]:
# Unifico los elementos que forman parte de surfistas en la columna "Activity" 

In [28]:
df_clean.loc[df_clean["Activity"].str.contains("Surfing"),"Activity"] = "Surfing"

In [29]:
df_clean.shape

(5033, 4)

In [30]:
# Filtro con las condiciones especificadas

In [31]:
df_clean=df_clean[(df_clean["Activity"] == "Surfing") & (df_clean["Country"] == "USA") & (df_clean["Area"] == "Florida")]

In [32]:
df_clean.shape

(363, 4)

#### TRATAMIENTO DEL CAMPO "DATE"

In [33]:
# Analizo como se configura el campo fecha 

In [34]:
df_clean["Month"]= df_clean["Date"].str.extract(r"([ADSMJNOF][a-z]{2})").astype(np.str)


In [35]:
print(set(df_clean.Month.values))

{'Oct', 'Jul', 'Apr', 'Jun', 'Jan', 'Feb', 'May', 'nan', 'Aug', 'Dec', 'Nov', 'Sep', 'Mar'}


In [36]:
# Analizo los valores nulos

df_clean[(df_clean["Month"]=="nan")]

,Date,Country,Area,Activity,Month
2460,1994,USA,Florida,Surfing,nan
2648,1990,USA,Florida,Surfing,nan
2908,1984,USA,Florida,Surfing,nan
3131,1978,USA,Florida,Surfing,nan
3491,1968,USA,Florida,Surfing,nan
6201,Beforer 1994,USA,Florida,Surfing,nan


In [37]:
# Elimino los registros de los cuales no tengo referencia al mes en el que ocurrió

df_clean=df_clean.drop(df_clean[df_clean.Month == "nan"].index)


#### ELIMINO LA COLUMNA "DATE" YA QUE PARA MI ANÁLISIS USARÉ LA COLUMNA "MONTH"

In [38]:
df_clean=df_clean.drop(["Date"], axis=1)

#### GUARDO EL ARCHIVO COMO .CSV

In [39]:
df_clean.to_csv("data/sharkattack_limpio.csv")